In this notebook i try multiple different methods to have more variance in inference

Uses Transformer without sigmoid output

## This uses already known data -> no true Testdata

## Get data

In [1]:
import torch
import numpy as np
import data_visualization.Visualization as visualization

# Check if GPU is available, set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [2]:
# Define Data parameters

sos_token = np.full((1, 24), 1)
pad_token = np.full((1, 24), 2)
pad_token = torch.tensor(pad_token, device=device)

dataset_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/jupyter_notebooks/evaluation/specific_test_midis/split_into_mid"
snapshot_intervall = 0.05

batch_size = 1
seq_length = 1024
stride = 128

test_size=0.1

In [3]:
from transformer_decoder_training.dataprep_transformer.prepare_dataloader_complete import prepare_dataset_as_dataloaders
from transformer_decoder_training.dataprep_transformer.prepare_dataloader_complete import prepare_dataset_as_single_loader

# Load Data

#train_loader, val_loader, test_loader = prepare_dataset_as_dataloaders(dataset_dir, snapshot_intervall, batch_size, seq_length, stride, test_size, sos_token)
train_loader = prepare_dataset_as_single_loader(dataset_dir, snapshot_intervall, batch_size, seq_length, stride, sos_token, shuffle=False)

Processed dataset (2/2): 100%|██████████| 2/2 [00:00<00:00,  5.90it/s]

Processed 2 of 2 files


## Get test sequence

In [4]:
from transformer_decoder_training.transformer_inference_eval import inference_and_visualize_1

# Get sequence
sequence = next(iter(train_loader))
print(sequence.shape)

context_seq, continuing_seq, original_seq = inference_and_visualize_1.prepare_sequence(sequence, 513)

torch.Size([1, 1025, 24])


## 6.1

Test model with BinaryCrossEntropy with logits loss

## Load model

In [5]:
# Define model Parameters

# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

# Transformer without sigmoid output
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_notebook_v6.1.pth"))
model.eval()

Transformer(
  (embedding): Linear(in_features=24, out_features=256, bias=True)
  (pos_emb): SinusoidalPosEmb()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ELU(alpha=1.0)
        (2): Linear(in_features=1024, out_features=256, bias=True)
      )
    )
  )
  (fc_out): Linear(in_features=256, out_features=24, bias=True)
)

## Inference

In [6]:
midi_save_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.1"

In [7]:
#Inference with threshold

from transformer_decoder_training.inference import inference_3

output_tokens, harmony_output_tokens, last_input_seq = inference_3.inference(model, context_seq, continuing_seq, 0.17, pad_token, device)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, last_input_seq, 0.05, midi_save_dir, "threshold_only.mid")


Tokens to generate: 512
Token after sigmoid:  tensor([[1.7756e-04, 2.3891e-04, 8.5692e-04, 9.4987e-01, 1.0383e-04, 8.0992e-03,
         1.0123e-03, 4.5589e-05, 2.1518e-03, 3.5198e-04, 2.7881e-02, 2.2402e-03,
         2.4761e-03, 4.0339e-04, 9.5440e-01, 2.2983e-02, 1.0295e-04, 8.4073e-03,
         1.0358e-02, 4.2998e-04, 8.3879e-03, 2.2442e-04, 2.7155e-02, 2.3312e-03]],
       device='cuda:0')
Token after sigmoid:  tensor([[2.0268e-04, 5.3392e-04, 1.1503e-03, 9.5135e-01, 9.1431e-05, 8.9648e-03,
         7.0780e-04, 9.0997e-05, 2.4528e-03, 4.9461e-04, 2.4702e-02, 2.0412e-03,
         2.4101e-03, 5.0277e-04, 9.6162e-01, 1.9623e-02, 1.6912e-04, 1.1510e-02,
         1.0533e-02, 5.0345e-04, 9.8536e-03, 2.4350e-04, 2.3403e-02, 4.1206e-03]],
       device='cuda:0')
Token after sigmoid:  tensor([[2.2068e-04, 6.9865e-04, 1.3698e-03, 9.5544e-01, 7.6818e-05, 6.1176e-03,
         4.7609e-04, 9.1483e-05, 2.7304e-03, 3.7228e-04, 3.1636e-02, 1.6123e-03,
         2.1538e-03, 3.7905e-04, 9.6051e-01, 2.0

In [8]:
# Inference with sampling
from transformer_decoder_training.inference import inference_4

output_tokens, harmony_output_tokens, last_input_seq = inference_4.inference_with_temperature_sampling(model, context_seq, continuing_seq, 0.7, pad_token, device)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, last_input_seq, 0.05, midi_save_dir, "temperature_sampling.mid")

Tokens to generate: 512
Token after temperature sampling:  tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], device='cuda:0')
Token after temperature sampling:  tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], device='cuda:0')
Token after temperature sampling:  tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], device='cuda:0')
Token after temperature sampling:  tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], device='cuda:0')
Token after temperature sampling:  tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], device='cuda:0')
Token after temperature sampling:  tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0.,

In [9]:
# inference with sampling + max allowed notes

output_tokens, harmony_output_tokens, last_input_seq = inference_4.inference_with_temperature_and_max_notes_sampling(model, context_seq, continuing_seq, 0.8, 0.25, pad_token, device, 3)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, last_input_seq, 0.05, midi_save_dir, "temperature_sampling_max_notes.mid")

Tokens to generate: 512
Token probabilities after temperature sampling:  tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], device='cuda:0')
Binary token after applying threshold and max notes constraint:  tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], device='cuda:0')
Token probabilities after temperature sampling:  tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], device='cuda:0')
Binary token after applying threshold and max notes constraint:  tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], device='cuda:0')
Token probabilities after temperature sampling:  tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], device='cuda:0')
Binary token after applying thresh

## 6.2

Test model with basic CrossEntropyLoss

In [10]:
# Define model Parameters

# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

# Transformer without sigmoid output
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_notebook_v6.2.pth"))
model.eval()

Transformer(
  (embedding): Linear(in_features=24, out_features=256, bias=True)
  (pos_emb): SinusoidalPosEmb()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ELU(alpha=1.0)
        (2): Linear(in_features=1024, out_features=256, bias=True)
      )
    )
  )
  (fc_out): Linear(in_features=256, out_features=24, bias=True)
)

In [11]:
midi_save_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.2"

#Inference with threshold
from transformer_decoder_training.inference import inference_3

output_tokens, harmony_output_tokens, last_input_seq = inference_3.inference(model, context_seq, continuing_seq, 0.17, pad_token, device)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, last_input_seq, 0.05, midi_save_dir, "threshold_only.mid")

# Inference with sampling
from transformer_decoder_training.inference import inference_4

output_tokens, harmony_output_tokens, last_input_seq = inference_4.inference_with_temperature_sampling(model, context_seq, continuing_seq, 0.7, pad_token, device)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, last_input_seq, 0.05, midi_save_dir, "temperature_sampling.mid")

# inference with sampling + max allowed notes
output_tokens, harmony_output_tokens, last_input_seq = inference_4.inference_with_temperature_and_max_notes_sampling(model, context_seq, continuing_seq, 0.8, 0.25, pad_token, device, 3)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, last_input_seq, 0.05, midi_save_dir, "temperature_sampling_max_notes.mid")

Tokens to generate: 512
Token after sigmoid:  tensor([[1.0000, 0.9961, 1.0000, 1.0000, 0.9989, 0.9919, 0.9927, 0.9912, 0.9999,
         0.9958, 0.0558, 1.0000, 0.9825, 0.9261, 1.0000, 1.0000, 0.9982, 0.9471,
         0.4735, 0.9986, 0.9497, 0.8275, 0.9770, 0.9999]], device='cuda:0')
Token after sigmoid:  tensor([[1.0000e+00, 9.9997e-01, 1.0000e+00, 1.0000e+00, 9.9999e-01, 9.9985e-01,
         9.9964e-01, 9.9994e-01, 1.0000e+00, 9.9997e-01, 3.2559e-04, 1.0000e+00,
         9.8720e-01, 9.7134e-01, 1.0000e+00, 1.0000e+00, 9.9997e-01, 8.9240e-01,
         1.5147e-01, 9.9938e-01, 8.1034e-01, 9.8125e-01, 9.8003e-01, 9.9987e-01]],
       device='cuda:0')
Token after sigmoid:  tensor([[1.0000e+00, 9.9998e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9987e-01,
         9.9972e-01, 9.9994e-01, 1.0000e+00, 9.9998e-01, 7.4244e-04, 1.0000e+00,
         9.8336e-01, 9.6845e-01, 1.0000e+00, 1.0000e+00, 9.9995e-01, 8.7559e-01,
         1.2241e-01, 9.9948e-01, 8.3926e-01, 9.8297e-01, 9.7790e-01, 9.9986e-01